In [ ]:
# ==========================================================
# 🔧 Configuration - Lecture et écriture Azure Data Lake
# ==========================================================

# Nom du compte de stockage
storage_account = "datalakemedallion"

# Widgets pour saisir les secrets (aucune clé dans le code !)
dbutils.widgets.text("AZURE_STORAGE_KEY", "")
dbutils.widgets.text("SQL_PASSWORD", "")

# Récupération des valeurs saisies
storage_key = dbutils.widgets.get("AZURE_STORAGE_KEY")
password = dbutils.widgets.get("SQL_PASSWORD")

# Noms des conteneurs et dossier de destination
container_source_name = "bronze"
container_destinataire_name = "silver"
folder_trsf = "transformations"

# Configuration Spark (connexion au Data Lake)
spark.conf.set(f"fs.azure.account.key.{storage_account}.blob.core.windows.net", storage_key)
destination_path = f"wasbs://{container_destinataire_name}@{storage_account}.blob.core.windows.net/{folder_trsf}"

print("✅ Connexion configurée avec succès.")


# ==========================================================
# 📥 Lecture des datasets depuis le conteneur Bronze
# ==========================================================
df_clients = spark.read.parquet(f"wasbs://{container_source_name}@{storage_account}.blob.core.windows.net/clients/")
df_comptes = spark.read.parquet(f"wasbs://{container_source_name}@{storage_account}.blob.core.windows.net/comptes/")
df_transactions = spark.read.parquet(f"wasbs://{container_source_name}@{storage_account}.blob.core.windows.net/transactions/")

print("✅ Lecture des données Bronze terminée.")


# ==========================================================
# 🧩 Transformation des données
# ==========================================================

# -- Clients
df_clients = (
    df_clients
    .withColumn("date_of_birth", df_clients["date_of_birth"].cast("date"))
    .withColumn("customer_since", df_clients["customer_since"].cast("date"))
)

# -- Comptes
df_comptes = (
    df_comptes
    .withColumn("balance", df_comptes["balance"].cast("double"))
    .withColumn("created_at", df_comptes["created_at"].cast("date"))
)

# -- Transactions
df_transactions = (
    df_transactions
    .withColumn("amount", df_transactions["amount"].cast("double"))
    .withColumn("transaction_date", df_transactions["transaction_date"].cast("date"))
)

# Suppression des doublons
df_transactions = df_transactions.dropDuplicates()

# Gestion des valeurs nulles
df_transactions = df_transactions.fillna(
    value="Inconnu",
    subset=[
        "transaction_id", "account_id", "customer_id",
        "transaction_type", "currency", "merchant_category",
        "status", "channel", "country", "transaction_date"
    ]
)
df_transactions = df_transactions.fillna({"amount": 0.0})

print("✅ Transformations terminées.")


# ==========================================================
# 💾 Écriture vers le conteneur Silver
# ==========================================================
df_clients.write.mode("overwrite").parquet(f"{destination_path}/clients_trsf")
df_comptes.write.mode("overwrite").parquet(f"{destination_path}/comptes_trsf")
df_transactions.write.mode("overwrite").parquet(f"{destination_path}/transactions_trsf")

print("🎉 --- Transformations terminées et données enregistrées dans Silver ---")


In [1]:
print("hello world")

hello world
